## 연구 보드판
- 일단 감정지수가 가격 변동에 유용한가를 봐야한다
즉 가격이아니라 가격 변동에 영향이다
- 가격변동을 기준으로 한다.
- 혹은 z-value 변동을 기준으로한다.
- 만약에 추세의 변화를 알려고 하면 어떻게 해야하나?

- X(t) = 가격
- Y(t) = 심리
- N(t) = 글 수
- I(t) = 인덱스

## I. 패키지 밎 자료 불러오기

In [1]:
!pip install pymysql
!pip install sqlalchemy
!pip install mysqlclient

     |████████████████████████████████| 43 kB 2.8 MB/s 
     |████████████████████████████████| 87 kB 6.9 MB/s 
  Created wheel for mysqlclient: filename=mysqlclient-2.1.0-cp37-cp37m-linux_x86_64.whl size=99970 sha256=f1961b3dac39b4f838455b19dc40c45c601a51bdee109b1eb158890bccf3d6fa
  Stored in directory: /root/.cache/pip/wheels/97/d4/df/08cd6e1fa4a8691b268ab254bd0fa589827ab5b65638c010b4
Successfully built mysqlclient


In [2]:
!pip install pyecharts
!pip install chart_studio

     |████████████████████████████████| 135 kB 25.2 MB/s 
     |████████████████████████████████| 130 kB 51.5 MB/s 
     |████████████████████████████████| 64 kB 3.6 MB/s 


In [3]:
import warnings
warnings.filterwarnings('ignore')
import time
import datetime
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import pymysql
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc

import itertools
import operator

import base64
from io import BytesIO

from datetime import datetime, timedelta

from pyecharts.charts import Bar
from pyecharts.charts import Line
import plotly.offline as plyo
import cufflinks
import plotly.express as px
import plotly.graph_objects as go

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# 불러오기
XRP = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/XRP_only.csv')

In [6]:
XRP.tail()

,Unnamed: 0,date,close_KRW,volume,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K,positive_ratio_E,positive_ratio_K
1521,1521,2021-11-27,1215.0,1.644314e+08,3142.0,14.0,6397.0,13.0,6.0,20.0,0.328902,0.333333
1522,1522,2021-11-28,1220.0,2.488932e+08,NaN,NaN,NaN,10.0,8.0,17.0,NaN,0.285714
1523,1523,2021-11-29,1230.0,6.381037e+06,NaN,NaN,NaN,8.0,3.0,10.0,NaN,0.380952
1524,1524,2021-11-30,1255.0,7.274336e+06,908.0,1.0,1934.0,4.0,3.0,2.0,0.319381,0.444444
1525,1525,2021-12-01,1250.0,6.126219e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


## II.전처리


### 1) 영어 제외하기, 날짜 수정 약 371일 데이터

In [7]:
XRP1 = XRP.drop(columns=['Unnamed: 0','positive_E','neutrual_E','negative_E','positive_ratio_E'])

In [8]:
xrp = XRP1[-371:-1]

In [9]:
xrp

,date,close_KRW,volume,positive_K,neutrual_K,negative_K,positive_ratio_K
1155,2020-11-26,600.0,1.706453e+09,45.0,25.0,37.0,0.420561
1156,2020-11-27,627.0,1.029245e+09,26.0,13.0,18.0,0.456140
1157,2020-11-28,688.0,8.124706e+08,13.0,6.0,10.0,0.448276
1158,2020-11-29,665.0,4.950927e+08,16.0,8.0,8.0,0.500000
1159,2020-11-30,720.0,6.329513e+08,20.0,15.0,19.0,0.370370
...,...,...,...,...,...,...,...
1520,2021-11-26,1295.0,1.024812e+07,6.0,1.0,14.0,0.285714
1521,2021-11-27,1215.0,1.644314e+08,13.0,6.0,20.0,0.333333
1522,2021-11-28,1220.0,2.488932e+08,10.0,8.0,17.0,0.285714
1523,2021-11-29,1230.0,6.381037e+06,8.0,3.0,10.0,0.380952


## III.  가격 D1,D2추가 , P/N P/(P+n+N)

In [10]:
xrp1 = xrp.loc[::-1]

In [11]:
xrp = xrp1.reset_index(drop=True)

In [12]:
xrp

,date,close_KRW,volume,positive_K,neutrual_K,negative_K,positive_ratio_K
0,2021-11-30,1255.0,7.274336e+06,4.0,3.0,2.0,0.444444
1,2021-11-29,1230.0,6.381037e+06,8.0,3.0,10.0,0.380952
2,2021-11-28,1220.0,2.488932e+08,10.0,8.0,17.0,0.285714
3,2021-11-27,1215.0,1.644314e+08,13.0,6.0,20.0,0.333333
4,2021-11-26,1295.0,1.024812e+07,6.0,1.0,14.0,0.285714
...,...,...,...,...,...,...,...
365,2020-11-30,720.0,6.329513e+08,20.0,15.0,19.0,0.370370
366,2020-11-29,665.0,4.950927e+08,16.0,8.0,8.0,0.500000
367,2020-11-28,688.0,8.124706e+08,13.0,6.0,10.0,0.448276
368,2020-11-27,627.0,1.029245e+09,26.0,13.0,18.0,0.456140


In [13]:
xrp['P/N'] = (xrp['positive_K'])/(xrp['negative_K'])
xrp['D(1)P/N(t)'] = -1*(xrp['P/N'].diff().shift(-1))/1
xrp['D(2)P/N(t)'] = -1*(xrp['D(1)P/N(t)'].diff().shift(-1))/1

In [14]:
xrp['D(1)X(t)'] = -1*(xrp['close_KRW'].diff().shift(-1))/1
xrp['D(2)X(t)'] = -1*(xrp['D(1)X(t)'].diff().shift(-1))/1

In [15]:
xrp['P/(P+N)'] = (xrp['positive_K'])/((xrp['negative_K'])+(xrp['positive_K']))
xrp['D(1)P/(P+N)(t)'] = -1*(xrp['P/(P+N)'].diff().shift(-1))/1
xrp['D(2)P/(P+N)(t)'] = -1*(xrp['D(1)P/(P+N)(t)'].diff().shift(-1))/1

감정지수

In [16]:
a=0.9
b=0.09
c=0.01
xrp['I_(P/N)(t)'] = a*(xrp['P/(P+N)'])+b*(xrp['D(1)P/(P+N)(t)'])+c*(xrp['D(2)P/(P+N)(t)'])

In [17]:
pd.options.display.max_rows = 40
pd.options.display.max_columns = 40

### 1) n 다음날 추론하기

In [18]:
xrp_n = xrp

In [19]:
xrp_n['close_KRW1'] = (xrp_n['close_KRW'].shift(1))
xrp_n['D(1)X(t)1'] = (xrp_n['D(1)X(t)'].shift(1))
xrp_n['D(2)X(t)1'] = (xrp_n['D(2)X(t)'].shift(1))

In [20]:
xrp_n

,date,close_KRW,volume,positive_K,neutrual_K,negative_K,positive_ratio_K,P/N,D(1)P/N(t),D(2)P/N(t),D(1)X(t),D(2)X(t),P/(P+N),D(1)P/(P+N)(t),D(2)P/(P+N)(t),I_(P/N)(t),close_KRW1,D(1)X(t)1,D(2)X(t)1
0,2021-11-30,1255.0,7.274336e+06,4.0,3.0,2.0,0.444444,2.000000,1.200000,0.988235,25.0,15.0,0.666667,0.222222,0.148148,0.621481,NaN,NaN,NaN
1,2021-11-29,1230.0,6.381037e+06,8.0,3.0,10.0,0.380952,0.800000,0.211765,0.273529,10.0,5.0,0.444444,0.074074,0.097643,0.407643,1255.0,25.0,15.0
2,2021-11-28,1220.0,2.488932e+08,10.0,8.0,17.0,0.285714,0.588235,-0.061765,-0.283193,5.0,85.0,0.370370,-0.023569,-0.117508,0.330037,1230.0,10.0,5.0
3,2021-11-27,1215.0,1.644314e+08,13.0,6.0,20.0,0.333333,0.650000,0.221429,1.792857,-80.0,-85.0,0.393939,0.093939,0.460606,0.367606,1220.0,5.0,85.0
4,2021-11-26,1295.0,1.024812e+07,6.0,1.0,14.0,0.285714,0.428571,-1.571429,-1.904762,5.0,10.0,0.300000,-0.366667,-0.408333,0.232917,1215.0,-80.0,-85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,2020-11-30,720.0,6.329513e+08,20.0,15.0,19.0,0.370370,1.052632,-0.947368,-1.647368,55.0,78.0,0.512821,-0.153846,-0.255295,0.445139,681.0,-39.0,-94.0
366,2020-11-29,665.0,4.950927e+08,16.0,8.0,8.0,0.500000,2.000000,0.700000,0.844444,-23.0,-84.0,0.666667,0.101449,0.127141,0.610402,720.0,55.0,78.0
367,2020-11-28,688.0,8.124706e+08,13.0,6.0,10.0,0.448276,1.300000,-0.144444,-0.372673,61.0,34.0,0.565217,-0.025692,-0.067820,0.505705,665.0,-23.0,-84.0
368,2020-11-27,627.0,1.029245e+09,26.0,13.0,18.0,0.456140,1.444444,0.228228,NaN,27.0,NaN,0.590909,0.042129,NaN,NaN,688.0,61.0,34.0


In [21]:
corr = xrp.corr(method='pearson')
print(corr)

                  close_KRW    volume  positive_K  neutrual_K  negative_K  \
close_KRW          1.000000 -0.063549   -0.011355   -0.082420   -0.147206   
volume            -0.063549  1.000000    0.770656    0.698618    0.742539   
positive_K        -0.011355  0.770656    1.000000    0.867660    0.901165   
neutrual_K        -0.082420  0.698618    0.867660    1.000000    0.896818   
negative_K        -0.147206  0.742539    0.901165    0.896818    1.000000   
positive_ratio_K   0.153014 -0.054527    0.074234   -0.246012   -0.190156   
P/N                0.127064 -0.151534   -0.118053   -0.226031   -0.301523   
D(1)P/N(t)        -0.006746 -0.022921    0.027085   -0.020952   -0.092030   
D(2)P/N(t)        -0.002299 -0.011272    0.026291    0.005927   -0.036250   
D(1)X(t)           0.086743  0.016324   -0.021024   -0.079727   -0.091353   
D(2)X(t)           0.000975 -0.025850   -0.062562   -0.020929    0.002853   
P/(P+N)            0.187679 -0.101706   -0.017865   -0.187026   -0.294807   

## IV. z-value 연구

In [22]:
n = [7,14,30,60,90,180]
crypto = [xrp]

for j in crypto :
  for i in n:
    j['MA{0}'.format(i)]= j['close_KRW'].rolling(window=i).mean().shift(-(i-1))

In [23]:
n = [7,14,30,60,90,180]
crypto = [xrp]
for j in crypto:
  for i in n :
    j['SD{0}(t)'.format(i)]= j['close_KRW'].rolling(window=i).std().shift(-(i-1))
  for i in n :
    j['MA{0}z(t)'.format(i)] = (j['close_KRW']-j['MA{0}'.format(i)])/(j['SD{0}(t)'.format(i)])

In [24]:
corr = xrp.corr(method='pearson')
print(corr)

                  close_KRW    volume  positive_K  neutrual_K  negative_K  \
close_KRW          1.000000 -0.063549   -0.011355   -0.082420   -0.147206   
volume            -0.063549  1.000000    0.770656    0.698618    0.742539   
positive_K        -0.011355  0.770656    1.000000    0.867660    0.901165   
neutrual_K        -0.082420  0.698618    0.867660    1.000000    0.896818   
negative_K        -0.147206  0.742539    0.901165    0.896818    1.000000   
positive_ratio_K   0.153014 -0.054527    0.074234   -0.246012   -0.190156   
P/N                0.127064 -0.151534   -0.118053   -0.226031   -0.301523   
D(1)P/N(t)        -0.006746 -0.022921    0.027085   -0.020952   -0.092030   
D(2)P/N(t)        -0.002299 -0.011272    0.026291    0.005927   -0.036250   
D(1)X(t)           0.086743  0.016324   -0.021024   -0.079727   -0.091353   
D(2)X(t)           0.000975 -0.025850   -0.062562   -0.020929    0.002853   
P/(P+N)            0.187679 -0.101706   -0.017865   -0.187026   -0.294807   

### 2) 일단 z-value 매력도 모이는 날 측정 - z밸류 정규화

*   z 밸류 상단 하단이 MA마다 다르다 그래서 정규화를 한번더 한다




In [25]:
n = [7,14,30,60,90,180]
crypto = [xrp]
for j in crypto:
  for i in n :
    j['MA{0}zMM(t)'.format(i)]=(j['MA{0}z(t)'.format(i)]-j['MA{0}z(t)'.format(i)].min())/(j['MA{0}z(t)'.format(i)].max()-j['MA{0}z(t)'.format(i)].min())

In [26]:
corr = xrp.corr(method='pearson')
print(corr['MA7zMM(t)'])

close_KRW      0.104057
volume         0.128495
positive_K     0.070641
neutrual_K    -0.057218
negative_K    -0.064894
                 ...   
MA14zMM(t)     0.850774
MA30zMM(t)     0.626416
MA60zMM(t)     0.490842
MA90zMM(t)     0.404919
MA180zMM(t)    0.176054
Name: MA7zMM(t), Length: 42, dtype: float64


### 3) z기준 I 통제 측정하기

In [27]:
# 감정지수 고려 없이
crypto = [xrp]
N = 0
D = 0
Xt = 0

MMa = 0.2
# for j in crypto:
for i in range(len(xrp)) :
    if (xrp['MA90zMM(t)'][i] <= MMa) and (xrp['MA180zMM(t)'][i] <= MMa) and (xrp['MA30zMM(t)'][i] <= MMa):
      print(xrp['date'][i])
      print('감정지수', xrp['I_(P/N)(t)'][i])
      a = xrp['close_KRW'][i-1]-xrp['close_KRW'][i]
      print('다음날 결과:', a, '\n')
      N += 1
      Xt += a
      if a >= 0:
        D += 1

print('승률 :' ,D/N)
print('누적 :' , Xt)

2021-07-20
감정지수 0.47170940170940173
다음날 결과: 49.0 

2021-07-19
감정지수 0.25246354952237304
다음날 결과: -38.0 

2021-06-26
감정지수 0.34904761904761905
다음날 결과: 29.0 

2021-06-25
감정지수 0.6088095238095238
다음날 결과: -1.0 

2021-06-23
감정지수 0.4356280027453672
다음날 결과: 40.0 

2021-06-22
감정지수 0.5770525432776634
다음날 결과: 101.0 

2021-06-21
감정지수 0.6160701945808329
다음날 결과: -79.0 

승률 : 0.5714285714285714
누적 : 101.0


In [ ]:
# 감정지수 하에
crypto = [xrp]
N = 0
D = 0
Xt = 0

MMa = 0.2
Ia = 0.5
# for j in crypto:
for i in range(len(xrp)) :
    if (xrp['MA90zMM(t)'][i] <= MMa) and (xrp['MA180zMM(t)'][i] <= MMa) and (xrp['MA30zMM(t)'][i] <= MMa) and (xrp['I_(P/N)(t)'][i] >= Ia):
      print(xrp['date'][i])
      print('감정지수', xrp['I_(P/N)(t)'][i])
      a = xrp['close_KRW'][i-1]-xrp['close_KRW'][i]
      print('다음날 결과:', a, '\n')
      N += 1
      Xt += a
      if a >= 0:
        D += 1

print('감정지수 통제 하 승률 :' ,D/N)
print('누적 :' , Xt)

2021-06-25
감정지수 0.6088095238095238
다음날 결과: -1.0 

2021-06-22
감정지수 0.5770525432776634
다음날 결과: 101.0 

2021-06-21
감정지수 0.6160701945808329
다음날 결과: -79.0 

감정지수 통제 하 승률 : 0.3333333333333333
누적 : 21.0


In [ ]:
# 감정지수 고려 없이
crypto = [xrp]
N = 0
D = 0
Xt = 0

MMa = 0.2
# for j in crypto:
for i in range(len(xrp)) :
    if(xrp['MA7zMM(t)'][i] <= MMa) and (xrp['MA14zMM(t)'][i] <= MMa):
      print(xrp['date'][i])
      print('감정지수', xrp['I_(P/N)(t)'][i])
      a = xrp['close_KRW'][i-1]-xrp['close_KRW'][i]
      print('다음날 결과:', a, '\n')
      N += 1
      Xt += a
      if a >= 0:
        D += 1

print('승률 :' ,D/N)
print('누적 :' , Xt)

2021-11-27
감정지수 0.3676060606060606
다음날 결과: 5.0 

2021-11-18
감정지수 0.5212197521148927
다음날 결과: 60.0 

2021-11-16
감정지수 0.5603921568627451
다음날 결과: 0.0 

2021-10-27
감정지수 0.5564746543778801
다음날 결과: 50.0 

2021-09-21
감정지수 0.45295238095238094
다음날 결과: 150.0 

2021-09-20
감정지수 0.4255238095238096
다음날 결과: -65.0 

2021-07-20
감정지수 0.47170940170940173
다음날 결과: 49.0 

2021-07-19
감정지수 0.25246354952237304
다음날 결과: -38.0 

2021-07-15
감정지수 0.31500000000000006
다음날 결과: -8.0 

2021-06-22
감정지수 0.5770525432776634
다음날 결과: 101.0 

2021-06-21
감정지수 0.6160701945808329
다음날 결과: -79.0 

2021-06-19
감정지수 0.44621153846153844
다음날 결과: 9.0 

2021-06-07
감정지수 0.5532539682539683
다음날 결과: 0.0 

2021-05-23
감정지수 0.3235312899106002
다음날 결과: 228.0 

2021-05-22
감정지수 0.42158082351229453
다음날 결과: -173.0 

2021-05-21
감정지수 0.4469445192566658
다음날 결과: -180.0 

2021-05-19
감정지수 0.4816843971631205
다음날 결과: 5.0 

2021-05-12
감정지수 0.4909285714285715
다음날 결과: 140.0 

2021-01-27
감정지수 0.36390529545151395
다음날 결과: 8.0 

2020-12-23
감정지수 0.3862279462046957
다음날

In [ ]:
# 감정지수 하에
crypto = [xrp]
N = 0
D = 0
Xt = 0

MMa = 0.2
Ia = 0.5
# for j in crypto:
for i in range(len(xrp)) :
    if (xrp['MA7zMM(t)'][i] <= MMa) and (xrp['MA14zMM(t)'][i] <= MMa) and (xrp['I_(P/N)(t)'][i] >= Ia) :
    # and (xrp['I_(P/N)(t)'][i] >= Ia):
      print(xrp['date'][i])
      print('감정지수', xrp['I_(P/N)(t)'][i])
      a = xrp['close_KRW'][i-1]-xrp['close_KRW'][i]
      print('다음날 결과:', a, '\n')
      N += 1
      Xt += a
      if a >= 0:
        D += 1

print('감정지수 통제 하 승률 :' ,D/N)
print('누적 :' , Xt)

2021-11-18
감정지수 0.5212197521148927
다음날 결과: 60.0 

2021-11-16
감정지수 0.5603921568627451
다음날 결과: 0.0 

2021-10-27
감정지수 0.5564746543778801
다음날 결과: 50.0 

2021-06-22
감정지수 0.5770525432776634
다음날 결과: 101.0 

2021-06-21
감정지수 0.6160701945808329
다음날 결과: -79.0 

2021-06-07
감정지수 0.5532539682539683
다음날 결과: 0.0 

감정지수 통제 하 승률 : 0.8333333333333334
누적 : 132.0


In [ ]:
# 감정지수 없이
crypto = [xrp]
N = 0
D = 0
Xt = 0

MMa = 0.2
Ia = 0.5
# for j in crypto:
for i in range(len(xrp)) :
    if (xrp['MA7zMM(t)'][i] <= MMa) : 
    # and (xrp['MA14zMM(t)'][i] <= MMa) and (xrp['I_(P/N)(t)'][i] >= Ia) :

      print(xrp['date'][i])
      print('감정지수', xrp['I_(P/N)(t)'][i])
      a = xrp['close_KRW'][i-1]-xrp['close_KRW'][i]
      print('다음날 결과:', a, '\n')
      N += 1
      Xt += a
      if a >= 0:
        D += 1

print('감정지수 없이 :' ,D/N)
print('누적 :' , Xt)

2021-11-27
감정지수 0.3676060606060606
다음날 결과: 5.0 

2021-11-18
감정지수 0.5212197521148927
다음날 결과: 60.0 

2021-11-17
감정지수 0.5023742540494458
다음날 결과: -55.0 

2021-11-16
감정지수 0.5603921568627451
다음날 결과: 0.0 

2021-10-27
감정지수 0.5564746543778801
다음날 결과: 50.0 

2021-09-28
감정지수 0.4767261904761904
다음날 결과: 40.0 

2021-09-21
감정지수 0.45295238095238094
다음날 결과: 150.0 

2021-09-20
감정지수 0.4255238095238096
다음날 결과: -65.0 

2021-09-08
감정지수 0.5196296296296297
다음날 결과: -15.0 

2021-09-07
감정지수 0.26978835978835974
다음날 결과: -30.0 

2021-08-26
감정지수 0.6629326923076924
다음날 결과: 125.0 

2021-08-24
감정지수 0.44251724137931037
다음날 결과: 35.0 

2021-08-03
감정지수 0.25672727272727275
다음날 결과: 4.0 

2021-07-20
감정지수 0.47170940170940173
다음날 결과: 49.0 

2021-07-19
감정지수 0.25246354952237304
다음날 결과: -38.0 

2021-07-17
감정지수 0.571610294117647
다음날 결과: 6.0 

2021-07-16
감정지수 0.15183333333333335
다음날 결과: -9.0 

2021-07-15
감정지수 0.31500000000000006
다음날 결과: -8.0 

2021-07-14
감정지수 0.7625000000000001
다음날 결과: -24.0 

2021-07-08
감정지수 0.3544444444444444
다음날 

In [ ]:
# 감정지수 통제
crypto = [xrp]
N = 0
D = 0
Xt = 0

MMa = 0.2
Ia = 0.5
# for j in crypto:
for i in range(len(xrp)) :
    if (xrp['MA7zMM(t)'][i] <= MMa) and (xrp['I_(P/N)(t)'][i] >= Ia) : 
    # and (xrp['MA14zMM(t)'][i] <= MMa) and (xrp['I_(P/N)(t)'][i] >= Ia) :

      print(xrp['date'][i])
      print('감정지수', xrp['I_(P/N)(t)'][i])
      a = xrp['close_KRW'][i-1]-xrp['close_KRW'][i]
      print('다음날 결과:', a, '\n')
      N += 1
      Xt += a
      if a >= 0:
        D += 1

print('감정지수 통제하 :' ,D/N)
print('누적 :' , Xt)

2021-11-18
감정지수 0.5212197521148927
다음날 결과: 60.0 

2021-11-17
감정지수 0.5023742540494458
다음날 결과: -55.0 

2021-11-16
감정지수 0.5603921568627451
다음날 결과: 0.0 

2021-10-27
감정지수 0.5564746543778801
다음날 결과: 50.0 

2021-09-08
감정지수 0.5196296296296297
다음날 결과: -15.0 

2021-08-26
감정지수 0.6629326923076924
다음날 결과: 125.0 

2021-07-17
감정지수 0.571610294117647
다음날 결과: 6.0 

2021-07-14
감정지수 0.7625000000000001
다음날 결과: -24.0 

2021-06-22
감정지수 0.5770525432776634
다음날 결과: 101.0 

2021-06-21
감정지수 0.6160701945808329
다음날 결과: -79.0 

2021-06-18
감정지수 0.8382727272727272
다음날 결과: -44.0 

2021-06-07
감정지수 0.5532539682539683
다음날 결과: 0.0 

2021-04-19
감정지수 0.5660681857460018
다음날 결과: 35.0 

2021-03-12
감정지수 0.5422431077694235
다음날 결과: 18.0 

감정지수 통제하 : 0.6428571428571429
누적 : 178.0


In [40]:
# 감정지수 없이
crypto = [xrp]
N = 0
D = 0
Xt = 0

MMa = 0.2
Ia = 0.5
# for j in crypto:
for i in range(len(xrp)) :
    if (xrp['MA14zMM(t)'][i] <= MMa) : 
    # and (xrp['MA14zMM(t)'][i] <= MMa) and (xrp['I_(P/N)(t)'][i] >= Ia) :

      print(xrp['date'][i])
      print('감정지수', xrp['I_(P/N)(t)'][i])
      a = xrp['close_KRW'][i-1]-xrp['close_KRW'][i]
      print('다음날 결과:', a, '\n')
      N += 1
      Xt += a
      if a >= 0:
        D += 1

print('감정지수 없이 :' ,D/N)
print('누적 :' , Xt)

2021-11-27
감정지수 0.3676060606060606
다음날 결과: 5.0 

2021-11-18
감정지수 0.5212197521148927
다음날 결과: 60.0 

2021-11-16
감정지수 0.5603921568627451
다음날 결과: 0.0 

2021-10-27
감정지수 0.5564746543778801
다음날 결과: 50.0 

2021-09-21
감정지수 0.45295238095238094
다음날 결과: 150.0 

2021-09-20
감정지수 0.4255238095238096
다음날 결과: -65.0 

2021-07-20
감정지수 0.47170940170940173
다음날 결과: 49.0 

2021-07-19
감정지수 0.25246354952237304
다음날 결과: -38.0 

2021-07-15
감정지수 0.31500000000000006
다음날 결과: -8.0 

2021-06-22
감정지수 0.5770525432776634
다음날 결과: 101.0 

2021-06-21
감정지수 0.6160701945808329
다음날 결과: -79.0 

2021-06-19
감정지수 0.44621153846153844
다음날 결과: 9.0 

2021-06-07
감정지수 0.5532539682539683
다음날 결과: 0.0 

2021-05-23
감정지수 0.3235312899106002
다음날 결과: 228.0 

2021-05-22
감정지수 0.42158082351229453
다음날 결과: -173.0 

2021-05-21
감정지수 0.4469445192566658
다음날 결과: -180.0 

2021-05-19
감정지수 0.4816843971631205
다음날 결과: 5.0 

2021-05-12
감정지수 0.4909285714285715
다음날 결과: 140.0 

2021-02-25
감정지수 0.5299261083743841
다음날 결과: -6.0 

2021-01-27
감정지수 0.36390529545151395
다음

In [35]:
# 감정지수 통제
crypto = [xrp]
N = 0
D = 0
Xt = 0

MMa = 0.2
Ia = 0.5
# for j in crypto:
for i in range(len(xrp)) :
    if (xrp['MA14zMM(t)'][i] <= MMa) and (xrp['I_(P/N)(t)'][i] >= Ia) : 
    # and (xrp['MA14zMM(t)'][i] <= MMa) and (xrp['I_(P/N)(t)'][i] >= Ia) :

      print(xrp['date'][i])
      print('감정지수', xrp['I_(P/N)(t)'][i])
      a = xrp['close_KRW'][i-1]-xrp['close_KRW'][i]
      print('다음날 결과:', a, '\n')
      N += 1
      Xt += a
      if a >= 0:
        D += 1

print('감정지수 통제하 :' ,D/N)
print('누적 :' , Xt)

2021-11-18
감정지수 0.5212197521148927
다음날 결과: 60.0 

2021-11-16
감정지수 0.5603921568627451
다음날 결과: 0.0 

2021-10-27
감정지수 0.5564746543778801
다음날 결과: 50.0 

2021-06-22
감정지수 0.5770525432776634
다음날 결과: 101.0 

2021-06-21
감정지수 0.6160701945808329
다음날 결과: -79.0 

2021-06-07
감정지수 0.5532539682539683
다음날 결과: 0.0 

2021-02-25
감정지수 0.5299261083743841
다음날 결과: -6.0 

감정지수 통제하 : 0.7142857142857143
누적 : 126.0


### 4) I 로만 측정하기

In [54]:
# 감정지수만
crypto = [xrp]
N = 0
D = 0
Xt = 0

MMa = 0.2
Ia = 0.65
# for j in crypto:
for i in range(len(xrp)) :
    if (xrp['I_(P/N)(t)'][i] >= Ia) :
    # or ( 0.62 >= xrp['I_(P/N)(t)'][i] >= 0.0) : 
    # and (xrp['MA14zMM(t)'][i] <= MMa) and (xrp['I_(P/N)(t)'][i] >= Ia) :

      print(xrp['date'][i])
      print('감정지수', xrp['I_(P/N)(t)'][i])
      a = xrp['close_KRW'][i-1]-xrp['close_KRW'][i]
      print('다음날 결과:', a, '\n')
      N += 1
      Xt += a
      if a >= 0:
        D += 1

print('감정지수 승률 :' ,D/N)
print('누적 :' , Xt)

2021-11-23
감정지수 0.6524864864864864
다음날 결과: -35.0 

2021-11-19
감정지수 0.6813210702341136
다음날 결과: 10.0 

2021-11-15
감정지수 0.7844047619047619
다음날 결과: -85.0 

2021-11-08
감정지수 0.6863636363636363
다음날 결과: -30.0 

2021-11-05
감정지수 0.7544201000238152
다음날 결과: -10.0 

2021-11-02
감정지수 0.6767251461988305
다음날 결과: 100.0 

2021-11-01
감정지수 0.6565497076023391
다음날 결과: 35.0 

2021-10-31
감정지수 0.7273830409356726
다음날 결과: -20.0 

2021-10-29
감정지수 0.6891725105189341
다음날 결과: -15.0 

2021-10-17
감정지수 0.661390977443609
다음날 결과: -10.0 

2021-10-09
감정지수 0.7100266796022313
다음날 결과: -10.0 

2021-10-08
감정지수 0.7182594824530308
다음날 결과: 120.0 

2021-10-06
감정지수 0.6523076923076923
다음날 결과: -10.0 

2021-10-04
감정지수 0.6532857142857144
다음날 결과: 30.0 

2021-09-29
감정지수 0.7248253968253969
다음날 결과: 25.0 

2021-09-23
감정지수 0.6694444444444445
다음날 결과: -65.0 

2021-09-22
감정지수 0.7274444444444446
다음날 결과: -25.0 

2021-09-14
감정지수 0.7673809523809524
다음날 결과: 15.0 

2021-08-28
감정지수 0.7773992673992675
다음날 결과: -10.0 

2021-08-26
감정지수 0.6629326923076924
다음